In [ ]:
import cv2 as cv
import matplotlib 
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
import datetime
from collections import namedtuple
from operator import itemgetter
from pprint import pformat
from numpy import inf

In [ ]:
#global variable
color = {'r': 0,'g':1,'b':2 }


def histogram(img):
    hist2 = {}
    h = img.shape[0]
    l = img.shape[1]
    for i in range(h):
        for j in range(l):
            s = str(img[i,j,0])+'_'+str(img[i,j,1])+'_'+str(img[i,j,2])
            if(s in hist2.keys()):
                hist2[s] = hist2[s] + 1;
            else:
                 hist2[s] = 1
    #print(hist2)                
    return hist2

def getScale(v,kbits):
    #f = round(255/pow(2,round(kbits/3)));
    #r = round(v[0] / f);
    #g = round(v[1] / f);
    #b = round(v[2] / (f));
    colorBits = round(kbits/3);
    mask = pow(2,(colorBits))-1;
    mask = mask << (8-colorBits);
    remainingBIts = (kbits - 2*colorBits)
    bmask = pow(2,remainingBIts)-1; 
    bmask = bmask << (8-remainingBIts)
    r = v[0] & mask;
    g = v[1] & mask;
    b = v[2] & bmask;
    #print(mask,'-',bmask)
    #print(f) 
    #print(v[0],v[1],v[2],'->',r,g,b)
    return [r,g,b];

def scaleDownImage(img,kbits):
    h = img.shape[0]
    l = img.shape[1]
    imgNew = 0 * img;
    for i in range(h):
        for j in range(l):
            imgNew[i,j] = getScale(img[i,j],kbits);
    #print(imgNew.shape)        
    return imgNew 

def generateColorMap(img):
    global color
    OriginalColorMapArray = []
    h = img.shape[0]
    l = img.shape[1] 
   
    for i in range(h):
        for j in range(l):
            v = img[i,j]
            newV = [v[color['r']],v[color['g']],v[color['b']],i,j]
            OriginalColorMapArray.append(newV)
              
                
    return OriginalColorMapArray  


def sortedBy(OriginalColorMapArray,dimension):
    global color
    radixDict = {};
    sortedColorBoxArray = [];
    for i in range(len(OriginalColorMapArray)):
        value = OriginalColorMapArray[i][color[dimension]]
        if OriginalColorMapArray[i][color[dimension]] in radixDict.keys():
            radixDict[value].append(OriginalColorMapArray[i])
        else: 
            radixDict[value]= [OriginalColorMapArray[i]]
    for i in range(256):
        if i in radixDict.keys():
            sortedColorBoxArray.extend(radixDict[i])
    return sortedColorBoxArray  


def findAvgColor(OriginalColorMapArray,newImageDirectMapping):
    #global count 
    #count = count+1
    global color
    localArray = np.array(OriginalColorMapArray)
    #print(localArray)
    rAvg = int(np.mean(localArray[:,color['r']]))
    gAvg = int(np.mean(localArray[:,color['g']]))
    bAvg = int(np.mean(localArray[:,color['b']]))
    global newImage 
    for i in range(len(OriginalColorMapArray)):
        newImageDirectMapping[OriginalColorMapArray[i][3],OriginalColorMapArray[i][4]] = [rAvg,gAvg,bAvg]
    return newImageDirectMapping,[rAvg,gAvg,bAvg]


def medianCut(OriginalColorMapArray,levelK,k,colorsPerBox,kColorsMap,directMappeddummyImage):
  
    
    
    levelK = levelK + 1
    localArray =  np.array(OriginalColorMapArray)
    rMin = np.min(localArray[:,color['r']])
    rMax = np.max(localArray[:,color['r']])
    gMin = np.min(localArray[:,color['g']])
    gMax = np.max(localArray[:,color['g']])
    bMin = np.min(localArray[:,color['b']])
    bMax = np.max(localArray[:,color['b']])
    rRange = rMax - rMin
    gRange = gMax - gMin
    bRange = bMax - bMin
    
    
    
    #if(len(OriginalColorMapArray)<=colorsPerBox):  
    if(levelK>k):   
        directMappeddummyImage,newColour = findAvgColor(OriginalColorMapArray,directMappeddummyImage) 
        #global kColorsMap
        kColorsMap.append(newColour)
        return  directMappeddummyImage,kColorsMap  
    
    mid = int((len(OriginalColorMapArray))/2); 
    if((bRange >= gRange) and (bRange >= rRange)):
        sortedColorBox = sortedBy(OriginalColorMapArray,'b');
        #print("lenoriginal",len(OriginalColorMapArray),"lensorted",len(sortedColorBox),"mid",mid)
        #midr = sortedColorBox[mid][color['r']]
         
        firsthalfBox = sortedColorBox[0:mid]
        secondhalfBox = sortedColorBox[mid:] 
        
        directMappeddummyImage,kColorsMap = medianCut(firsthalfBox,levelK,k,colorsPerBox,kColorsMap,directMappeddummyImage)
        directMappeddummyImage,kColorsMap = medianCut(secondhalfBox,levelK,k,colorsPerBox,kColorsMap,directMappeddummyImage)
        
    elif(gRange >= rRange and gRange >= bRange):
        sortedColorBox = sortedBy(OriginalColorMapArray,'g');
        #midg = sortedColorBox[mid][color['g']]
        firsthalfBox = sortedColorBox[0:mid]
        secondhalfBox = sortedColorBox[mid:] 
        directMappeddummyImage,kColorsMap = medianCut(firsthalfBox,levelK,k,colorsPerBox,kColorsMap,directMappeddummyImage)
        directMappeddummyImage,kColorsMap = medianCut(secondhalfBox,levelK,k,colorsPerBox,kColorsMap,directMappeddummyImage)
    
    else:
        sortedColorBox = sortedBy(OriginalColorMapArray,'r');
        #midb = sortedColorBox[mid][color['b']]
        firsthalfBox = sortedColorBox[0:mid]
        secondhalfBox = sortedColorBox[mid:] 
        directMappeddummyImage,kColorsMap = medianCut(firsthalfBox,levelK,k,colorsPerBox,kColorsMap,directMappeddummyImage)
        directMappeddummyImage,kColorsMap = medianCut(secondhalfBox,levelK,k,colorsPerBox,kColorsMap,directMappeddummyImage)
    return  directMappeddummyImage,kColorsMap   



class Node(namedtuple("Node", "location left_child right_child")):
    def __repr__(self):
        return pformat(list(self))

def kdtree(point_list, depth: int = 0):
    if not point_list:
        return None

    k = len(point_list[0])  # assumes all points have the same dimension
    # Select axis based on depth so that axis cycles through all valid values
    axis = depth % k

    # Sort point list by axis and choose median as pivot element
    point_list.sort(key=itemgetter(axis))
    median = len(point_list) // 2

    # Create node and construct subtrees
    return Node(
        location=point_list[median],
        left_child=kdtree(point_list[:median], depth + 1),
        right_child=kdtree(point_list[median + 1 :], depth + 1),
    )
def findNearest(tree,point,minDist,closestPoint,depth):
        if(tree is None):
            return minDist,closestPoint
        else:
            x = tree.location
            v = point
            k = len(point)
            getdistance = 0
            for i in range(k):
                getdistance = getdistance + ((x[i]-v[i])**2.0)    
            getdistance = abs((getdistance)**0.5)
            #getdistance = abs((((x[0] - v[0]))**2.0 + ((x[1]-v[1]))**2.0 + ((x[2]-v[2]))**2.0 )**0.5)
            if getdistance < minDist:
                minDist = getdistance
                closestPoint = tree.location
            axis = depth % k
            if((tree.location)[axis] > point[axis]):
                return findNearest(tree.left_child,point,minDist,closestPoint,depth+1)
            elif((tree.location)[axis] < point[axis]): 
                return findNearest(tree.right_child,point,minDist,closestPoint,depth+1)
            else:
                if(tree.left_child):
                     minDist,closestPoint=findNearest(tree.left_child,point,minDist,closestPoint,depth+1)
                        
                if(tree.right_child):
                    minDistr,closestPointr=findNearest(tree.right_child,point,minDist,closestPoint,depth+1) 
                    if(minDist < minDistr):
                        minDist = minDistr
                        closestPoint = closestPointr
                    
                return minDist,closestPoint

            
#def redraw(img,newColours):
def redraw(img,tree):
    h = img.shape[0]
    l = img.shape[1]
    imgNew = 0 * img;
    for i in range(h):
        for j in range(l):
            #imgNew[i,j] = getColour(newColours,img[i,j]);
            minDist,point = findNearest(tree,img[i,j],inf,[],0)
            imgNew[i,j] = point
    #print(imgNew.shape)        
    return imgNew   

def limitV(vec):
    for i in range(len(vec)):
        if(vec[i]>255):
            vec[i] = 255
        if(vec[i]<0):
            vec[i] = 0
    return vec    
        

def redrawWithFloydAndStienBerg(imgOriginal,tree):
    h = imgOriginal.shape[0]
    l = imgOriginal.shape[1]
    k = len(imgOriginal[0,0])
    img = 1 * imgOriginal
    imgNew = 0 * img;
    for i in range(h):
        for j in range(l):
            minDist,point = findNearest(tree,img[i,j],inf,[],0)
            imgNew[i,j] = point
            error = []
            x = img[i,j]
            v = imgNew[i,j]
            for q in range(k):
                error.append((int(x[q]) - int(v[q]))) 
            #if(error != [0,0,0]):
            #print(x,"-",error,"-",v)    
            if((j+1) < l):
                #print(img[i,j+1])
                img[i,j+1] = limitV(np.add(np.array(img[i,j+1],dtype="int") , np.array([round(e*(3.0/8)) for e in error],dtype="int"),dtype="int").tolist())
                #print(img[i,j+1])
                #return imgNew
            if(i+1<h):     
                img[i+1,j] = limitV(np.add(np.array(img[i+1,j],dtype="int") , np.array([round(e*(3.0/8)) for e in error],dtype="int"),dtype="int").tolist())
            if(i+1<h and j+1<l):
                img[i+1,j+1] = limitV(np.add(np.array(img[i+1,j+1],dtype="int") , np.array([round(e*(1.0/4))  for e in error],dtype="int"),dtype="int").tolist())
    #print(imgNew.shape)        
    return imgNew


def runall(filename,k):
    imCv1 = cv.imread(filename)
    img = cv.cvtColor(imCv1, cv.COLOR_BGR2RGB)
    OriginalColorMap = generateColorMap(img)
    numNewColors = pow(2,k)
    colorsPerBox = int(math.ceil(len(OriginalColorMap))/numNewColors);
    newColourMap = []
    directMappeddummyImage = 0 * img.copy()
    newImageDirectMapping,newColourMap = medianCut(OriginalColorMap,0,k,colorsPerBox,newColourMap,directMappeddummyImage)
    print("OriginalColourMap Length :",len(OriginalColorMap),", New Color Map Length",len(newColourMap))
    tree = kdtree(newColourMap)
    medianCutImage = redraw(img,tree)
    medianCutDitherFloydImage = redrawWithFloydAndStienBerg(img,tree)
    f, axArr = plt.subplots(nrows=2,ncols=2,figsize=(50,50))
    plt.sca(axArr[0][0]); 
    plt.imshow(img); 
    plt.title('Original Image',fontsize=50)
    plt.sca(axArr[0][1]); 
    plt.imshow(newImageDirectMapping); 
    plt.title('k('+str(2**k)+')-Median Cut Direct Mapped',fontsize=50)
    plt.sca(axArr[1][0]); 
    plt.imshow(medianCutImage); 
    plt.title('k('+str(2**k)+')-Median Cut kdtree Mapped',fontsize=50)
    plt.sca(axArr[1][1]); 
    plt.imshow(medianCutDitherFloydImage); 
    plt.title('k('+str(2**k)+')-Median Cut kdtree Mapped and Floyed Dither',fontsize=50)
    plt.show()
    return img.copy(),medianCutImage,medianCutDitherFloydImage,tree

In [ ]:
originalIamge,medianCutImage,medianCutDitherFloydImage,colourTree = runall('pamela.jpg',3)

In [ ]:
originalIamge,medianCutImage,medianCutDitherFloydImage,colourTree = runall('pamela.jpg',4)

In [ ]:
originalIamge,medianCutImage,medianCutDitherFloydImage,colourTree = runall('pamela.jpg',6)

In [ ]:
originalIamge,medianCutImage,medianCutDitherFloydImage,colourTree = runall('pamela.jpeg',8)

In [ ]:
originalIamge,medianCutImage,medianCutDitherFloydImage,colourTree = runall('pamela.jpg',12)